# Apple detection based on YOLO algorithm

## Importation of python libraries

In [1]:
#select an image to display
from pathlib import Path
import csv
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import matplotlib.patches as patches
import os
import pandas as pd
import numpy as np
import imageio
import fnmatch
import random
import cv2 
import json
import random
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from IPython.display import Image as IpImage
from IPython.display import display
from PIL import Image as PilImage
from sklearn.cluster import KMeans
from skimage.io import imread, imshow, concatenate_images
from skimage import color
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from skimage.exposure import equalize_adapthist
import tensorflow as tf
from itertools import chain
from tqdm import tqdm
import h5py
import cv2 
import pandas as pd
import matplotlib.pyplot as plt 
import os
import imageio
import subprocess

## Fonctions

In [3]:
def size(data_path):
    #"Measure the size of dataset"
    txt_idx=0
    FILE=pd.read_csv(data_path,header=None)
    FILE=np.array(FILE)
    for filename in FILE:
            txt_idx=txt_idx+1     
    return  txt_idx  


def Visualizing_images(path,idx):
  #"Visualise images from a dataset txt file"
  print("Image n",idx)
  i=idx
  FILE=pd.read_csv(path,header=None)
  FILE=np.array(FILE)
  opt=list(FILE[i])
  opt=" ".join(opt)
  opt=opt.split('.')
  path_im=opt[0]+'.jpg'
  path_txt=opt[0]+'.txt'

  fig, ax = plt.subplots()
  fig.set_size_inches(15, 20)
  ax.set_axis_off()

  img  = imageio.imread(path_im)
  ax.imshow(img)
  height, width = img.shape[:2]
  plt.title(os.path.basename(path_im))
  df = pd.read_csv(path_txt, sep=" ",header=None)
  df=pd.DataFrame(df.values,columns=["Label", "x", "y", "width", "height"])
  for x, y, w, h in zip( df.x, df.y, df.width, df.height):
          x *= width
          y *= height
          w *= width
          h *= height
          x1 = int(x - w / 2 + 1)
          x2 = int(x1 + w)
          y1 = int(y - h / 2 + 1)
          y2 = int(y1 + h)
          ax.add_patch(patches.Rectangle((int(x1), int(y1)), w, h, linewidth=5, edgecolor='r', facecolor="none"))
  plt.show()


def Generate_Pred_Result(path_pred,path_test):
        #Generating a dataframe containing IMG ID,Confidence score, xmin,ymin,xmax,ymax"
        Bounding_boxes_doc=pd.read_json(path_pred)
        Template=path_test
        obj = Bounding_boxes_doc["objects"].apply(pd.Series)
        file = Bounding_boxes_doc["filename"].apply(pd.Series)
        Array=np.zeros((1,6))
        for i,l in zip(range(obj.shape[0]),range(len(file[0]))):
          for k in range(obj.shape[1]):
            if isinstance(obj[k][i], float):
              continue
            else:
              data_items = obj[k][i].items()
              data_list = list(data_items)
              df = pd.DataFrame(data_list,index=['class_id', 'name', 'relative_coordinates', 'confidence'])
              #############################################################################################
              confidence=df.loc['confidence'].items()
              confidence_list = list(confidence)
              cf=list(confidence_list[1])
              Confidence_score=cf[1]
              #############################################################################################
              img=cv2.imread(file[0][i])
              Height=img.shape[0]
              Width=img.shape[1]
              image_ID=file[0][i].replace(Template,'')
              Filename=image_ID.split('.')[0]
              #############################################################################################
              relative_coordinates=df.loc['relative_coordinates'].items()
              relative_coordinates_list = list(relative_coordinates)
              xywh=list(relative_coordinates_list[1][1].items())
              x=list(xywh[0])
              x=x[1]
              y=list(xywh[1])
              y=y[1]
              w=list(xywh[2])
              w=w[1]
              h=list(xywh[3])
              h=h[1]
              #############################################################################################
              x *= Width
              y *= Height
              w *= Width
              h *= Height
              x1 = int(x - w / 2 + 1)
              x2 = int(x1 + w)
              y1 = int(y - h / 2 + 1)
              y2 = int(y1 + h)
              Xmin=x1
              Ymin=y1
              Xmax=Xmin+w
              Ymax=Ymin+h
              #############################################################################################
              line_score=[Confidence_score,Xmin,Ymin,Xmax,Ymax]
              #############################################################################################
              line_score.insert(0,Filename)
              #############################################################################################
              Array=np.vstack((Array,line_score))
        s=Array[1:,:]
        df = pd.DataFrame(s, columns=['imageid','confidence', 'x','y', 'w','h'])
        return df

def Generate_Pred_Result_255(path_pred,path_test):
        #Generating a dataframe containing IMG ID,Confidence score, xmin,ymin,xmax,ymax"
        Bounding_boxes_doc=pd.read_json(path_pred)
        Template=path_test
        obj = Bounding_boxes_doc["objects"].apply(pd.Series)
        file = Bounding_boxes_doc["filename"].apply(pd.Series)
        Array=np.zeros((1,6))
        for i,l in zip(range(obj.shape[0]),range(len(file[0]))):
          for k in range(obj.shape[1]):
            if isinstance(obj[k][i], float):
              continue
            else:
              data_items = obj[k][i].items()
              data_list = list(data_items)
              df = pd.DataFrame(data_list,index=['class_id', 'name', 'relative_coordinates', 'confidence'])
              #############################################################################################
              confidence=df.loc['confidence'].items()
              confidence_list = list(confidence)
              cf=list(confidence_list[1])
              Confidence_score=cf[1]
              #############################################################################################
              img=cv2.imread(file[0][i])
              Height=256
              Width=256
              image_ID=file[0][i].replace(Template,'')
              Filename=image_ID.split('.')[0]
              #############################################################################################
              relative_coordinates=df.loc['relative_coordinates'].items()
              relative_coordinates_list = list(relative_coordinates)
              xywh=list(relative_coordinates_list[1][1].items())
              x=list(xywh[0])
              x=x[1]
              y=list(xywh[1])
              y=y[1]
              w=list(xywh[2])
              w=w[1]
              h=list(xywh[3])
              h=h[1]
              #############################################################################################
              x *= Width
              y *= Height
              w *= Width
              h *= Height
              x1 = int(x - w / 2 + 1)
              x2 = int(x1 + w)
              y1 = int(y - h / 2 + 1)
              y2 = int(y1 + h)
              Xmin=x1
              Ymin=y1
              Xmax=Xmin+w
              Ymax=Ymin+h
              #############################################################################################
              line_score=[Confidence_score,Xmin,Ymin,Xmax,Ymax]
              #############################################################################################
              line_score.insert(0,Filename)
              #############################################################################################
              Array=np.vstack((Array,line_score))
        s=Array[1:,:]
        df = pd.DataFrame(s, columns=['imageid','confidence', 'x','y', 'w','h'])
        return df

def submit(path_pred,path_test,Name,NMS):
        #"converting the dataframe exported in results to a submission file format"
        Template=path_test
        df=Generate_Pred_Result_255(path_pred,path_test)
        df['Values'] = df[df.columns[1:]].values.tolist()
        df = df.drop(['confidence','x','y','w','h'], axis=1)
        if NMS==True:
          print('1')
        fieldnames = ['imageid', 'Values']
        Folder=path_test+Name
        with open(Folder, 'w', newline='') as f:  # You will need 'wb' mode in Python 2.x
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for i,img in zip(df['Values'],df['imageid']):
                [cs,xmin,ymin,xmax,ymax]=i
                new_box=[float(cs),float(xmin),float(ymin),float(xmax),float(ymax)]
                #new_bbox.append(new_box)
                writer.writerow({'imageid':img,'Values': new_box})
       

def visualizepred(Path,index):
       #"Visualise images after prediction using Prediction.csv"
        Bounding_boxes_doc= pd.read_json(Path)
        Bounding_boxes_doc.head()
        obj = Bounding_boxes_doc["objects"].apply(pd.Series)
        file = Bounding_boxes_doc["filename"].apply(pd.Series)

        fig, ax = plt.subplots()
        fig.set_size_inches(15, 20)
        ax.set_axis_off()

        objects=obj.loc[index,:].dropna()
        images=file.loc[index,:].dropna()
        path=images[0]
        img=imageio.imread(path)
        print(os.path.basename(path))
        plt.title(os.path.basename(path))
        height,width = img.shape[:2]
        ax.imshow(img)
        print("number of bounding boxes:", len(objects))
        for i in range(len(objects)):
            d=objects.loc[i]
            m=d["relative_coordinates"]
            x =m["center_x"]
            y=m["center_y"]
            w=m["width"]
            h=m["height"]
            x *= width
            y *= height
            w *= width
            h *= height
            x1 = int(x - w / 2 + 1)
            x2 = int(x1 + w)
            y1 = int(y - h / 2 + 1)
            y2 = int(y1 + h)
            ax.add_patch(patches.Rectangle((int(x1), int(y1)), w, h, linewidth=5, edgecolor='r', facecolor="none"))
        plt.show()

def create_txt_files(current_dir,s):
  #Create txt files of data
  DIR=current_dir+s+'/'
  file_train = open(current_dir+s+'.txt', 'w')
  for img_filename in os.listdir(DIR):
      if img_filename.endswith(".png") or img_filename.endswith(".jpg") or img_filename.endswith(".JPG"):
        file_train.write(DIR + img_filename+ "\n")

## Inference

Original code to run in terminal

Folder_darknet='C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/darknet/'

os.chdir(Folder_darknet)

!chmod +x ./darknet

!./darknet detector test "C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/obj.data" "C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/yolov4-tiny-test.cfg" "C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/backup/yolov4-tiny-train_best.weights" -thresh 0.20 -dont_show -out "C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/Prediction.csv" < "C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/test.txt"

System detected : Windows

In [8]:
Folder_darknet='C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/darknet/'
os.chdir(Folder_darknet)

subprocess.run([".\darknet","detector","test","C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/obj.data","C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/yolov4-tiny-test.cfg","C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/backup/yolov4-tiny-train_best.weights","-thresh","0.20","-dont_show","-out","C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/Prediction.csv","<","C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/test.txt"],shell=True)

CompletedProcess(args=['.\\darknet', 'detector', 'test', 'C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/obj.data', 'C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/yolov4-tiny-test.cfg', 'C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/backup/yolov4-tiny-train_best.weights', '-thresh', '0.20', '-dont_show', '-out', 'C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/Prediction.csv', '<', 'C:/Users/User/OneDrive - Université de Bourgogne/Documents/Stage/yolo/main_folder/Model_ALL_views/test.txt'], returncode=1)